In [47]:


from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")
import os 
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [48]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec("aws", "us-east-1"))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\sgale\miniconda3\envs\rateMyProfessor\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
  File "C:\Users\sgale\AppData\Local\Temp\ipykernel_12572\2931132556.py", line 2, in <module>
    pc.create_index(name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec("aws", "us-east-1"))
  File "c:\Users\sgale\miniconda3\envs\rateMyProfessor\Lib\site-packages\pinecone\control\pinecone.py", line 384, in create_index
    api_instance.create_index(
  File "c:\Users\sgale\miniconda3\envs\rateMyProfessor\Lib\site-packages\pinecone\core\openapi\shared\api_client.py", line 761, in __call__
    return self.callable(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sgale\miniconda3\envs\rateMyProfessor\Lib\site-packages\pinecone\core\openapi\control\api\manage_indexes_api.py", line 273, in __create_index
    return self.call_with_http_info(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

In [ ]:
import json 
data = json.load(open("reviews.json"))
data['reviews']
len(data['reviews'])

20

In [ ]:
processed_data = [] 
client = OpenAI(
        api_key= os.getenv("OPENAI_API_KEY")
)

for review in data['reviews']:
    response = client.embeddings.create(
        input=review["review"], 
        model='text-embedding-3-small'
    )
    embeddings = response.data[0].embedding 
    processed_data.append({ 
        "values" : embeddings, 
        "id": review["professor"], 
        "metadata" : {
            "review" : review["review"],
            "subject" : review["subject"],
            "stars" : review["stars"] 
        }
        })

In [ ]:
index = pc.Index('rag')
index.upsert( 
            vectors=processed_data, 
            namespace="ns1"
             )

{'upserted_count': 20}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}